# Описание

В этом ноутбуке реализована очистка и предварительная обработка данных к разведочному анализу.

Удалены дублирующиеся строки и явно ошибочные данные. Заполнены пропуски в данных. Некоторые колонки представлены в удобном виде, отформатированы или удалены



**Описание датасета**

Данные о характеристиках и ценах автомобилей с пробегом собранные с сайта [olx.ua](https://olx.ua).

* **id** - id объявления
* **created_time** - дата и время создания объявления
* **car_state_type** - тип объявления о продаже
* **cleared_customs** - растаможен ли автомобиль
* **brand** - марка автомобиля
* **model** - модель автомобиля
* **motor_year** - год выпуска
* **motor_mileage_thou** - пробег авто, тыс. км.
* **car_body** - тип кузова
* **color** - код цвета кузова
* **transmission_type** - тип коробки передач
* **drive_type** - привод
* **fuel_type** - тип топлива
* **motor_engine_size_litre** - объём двигателя в литрах
* **exterior_condition** - внешнее состояние
* **condition** - список значений состояния 
* **region** - код области или региона
* **lat** - широта
* **lon** - долгота
* **price** - цена (целевая переменная)

# 1. Загрузка данных и вывод информации

Выведем следующую информацию: 
-   длину и количество колнок в датафрейме,
-   общую информацию о колонках,
-   первые несколько записей,
-   описание данных в числовых столбцах,
-   уникальные значения в столбцах.



In [ ]:
!rm -rf sample_data
import numpy as np
import pandas as pd

!rm cars.csv
!wget https://gitlab.com/glebtutik/car_price_estimation_data/-/raw/main/cars.csv

--2023-02-28 05:57:59--  https://gitlab.com/glebtutik/car_price_estimation_data/-/raw/main/cars.csv
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27362640 (26M) [text/plain]
Saving to: ‘cars.csv’

cars.csv            100%[===================>]  26.09M   146MB/s    in 0.2s    

2023-02-28 05:57:59 (146 MB/s) - ‘cars.csv’ saved [27362640/27362640]



In [ ]:
df = pd.read_csv("cars.csv")
df.shape

(127439, 20)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127439 entries, 0 to 127438
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       127439 non-null  int64  
 1   created_time             127439 non-null  object 
 2   car_state_type           127420 non-null  object 
 3   cleared_customs          127435 non-null  object 
 4   brand                    127439 non-null  object 
 5   model                    127206 non-null  object 
 6   motor_year               127433 non-null  float64
 7   motor_mileage_thou       84065 non-null   float64
 8   car_body                 127412 non-null  object 
 9   color                    127412 non-null  float64
 10  transmission_type        127413 non-null  object 
 11  drive_type               127415 non-null  object 
 12  fuel_type                127412 non-null  object 
 13  motor_engine_size_litre  76264 non-null   float64
 14  exte

In [ ]:
df.head()

,id,created_time,car_state_type,cleared_customs,brand,model,motor_year,motor_mileage_thou,car_body,color,transmission_type,drive_type,fuel_type,motor_engine_size_litre,exterior_condition,condition,region,lat,lon,price
0,780257891,2023-01-22T22:42:27+02:00,with_mileage,yes,Opel,corsa,2011.0,170.0,hatchback,6.0,Механічна,front,Дизель,1.3,major_fixes_needed,['after-an-accident'],lv,49.90548,24.09005,100.000000
1,780828675,2023-01-27T13:54:49+02:00,with_mileage,yes,ВАЗ,2106,1992.0,285.0,sedan,3.0,Механічна,back,Бензин,1.6,minor_scratches,['needs_engine_repair'],lv,50.09947,24.34957,81.603786
2,768317392,2022-10-19T15:45:24+03:00,with_mileage,yes,ЗАЗ,968,1992.0,NaN,sedan,1.0,Механічна,back,Бензин,10.0,major_fixes_needed,['not-on-the-move'],chk,49.43061,32.63275,81.603786
3,779502018,2023-01-16T22:31:33+02:00,with_mileage,yes,ВАЗ,2101,1979.0,NaN,sedan,1.0,Механічна,back,Бензин,NaN,major_fixes_needed,['needs_body_repair'],dnp,48.40500,35.01300,68.003155
4,780355459,2023-01-23T19:08:53+02:00,with_mileage,yes,ВАЗ,2101,1972.0,500.0,sedan,3.0,Механічна,back,Бензин,1.5,minor_fixes_needed,['fine_condition'],rov,51.33987,26.60507,54.402524


In [ ]:
df.describe()

,id,motor_year,motor_mileage_thou,color,motor_engine_size_litre,lat,lon,price
count,1.274390e+05,127433.000000,84065.000000,127412.000000,76264.000000,127439.000000,127439.000000,127439.000000
mean,7.660363e+08,2003.684517,266.907512,4.569829,1.945524,49.252705,30.019169,6382.463414
std,3.166428e+07,9.984954,357.731602,3.926583,0.816487,1.278902,4.137602,6346.610153
min,8.875286e+07,1930.000000,0.000000,1.000000,0.100000,44.962800,22.160770,27.271141
25%,7.609161e+08,1999.000000,141.000000,2.000000,1.500000,48.450040,26.125570,2200.000000
50%,7.773618e+08,2006.000000,209.000000,4.000000,1.800000,49.406110,30.502430,4800.000000
75%,7.812071e+08,2011.000000,280.000000,6.000000,2.000000,50.356840,33.392000,8450.000000
max,7.855604e+08,2025.000000,3000.000000,25.000000,10.000000,52.260000,40.129010,100000.000000


Уникальные значения:


In [ ]:
for column in df.columns:
    print(f"\n\n_____________ \"{column}\" _____________\n")

    uniques = df[column].unique()
    len_uniques = len(uniques)

    if len_uniques <= 100:
        try:
            print(np.sort(uniques))
        except:
            print(uniques)
    else:
        print(f"A lot of values to print ({len_uniques})\n")



_____________ "id" _____________

A lot of values to print (127439)



_____________ "created_time" _____________

A lot of values to print (126711)



_____________ "car_state_type" _____________

['with_mileage' 'to_order' nan 'new']


_____________ "cleared_customs" _____________

['yes' 'no' 'in_future' nan]


_____________ "brand" _____________

['Acura' 'Alfa Romeo' 'Audi' 'BMW' 'BMW-Alpina' 'BYD' 'Bentley'
 'Brilliance' 'Buick' 'Cadillac' 'Chana' 'Chery' 'Chevrolet' 'Chrysler'
 'Citroen' 'Dacia' 'Dadi' 'Daewoo' 'Daihatsu' 'Dodge' 'FAW' 'Fiat' 'Ford'
 'GMC' 'Geely' 'Great Wall' 'Groz' 'Honda' 'Hummer' 'Hyundai' 'Infiniti'
 'Isuzu' 'Iveco' 'JAC' 'Jaguar' 'Jeep' 'Kia' 'Lancia' 'Land Rover' 'Lexus'
 'Lifan' 'Lincoln' 'Lotus' 'MG' 'MINI' 'Maserati' 'Maybach' 'Mazda'
 'McLaren' 'Mercedes-Benz' 'Mercury' 'Mitsubishi' 'Nissan' 'Oldsmobile'
 'Opel' 'Peugeot' 'Pontiac' 'Porsche' 'Proton' 'Ram' 'Ravon' 'Renault'
 'Rolls-Royce' 'Rover' 'Saab' 'Samand' 'Samsung' 'Seat' 'Shelby' 'Skoda'
 'S

# 2. Предварительная обработка

### Удалим строки
 
*   где год выпуска авто больше 2022 или не указан (27 шт).
*   где не указан тип трансмиссии (21 шт).
*   содержащие новые авто и доступные только под заказ (540 шт).
*   содержащие авто (кроме электрокаров) с объёмом двигателя меньше 0.5 л или больше 6.0 л (580 шт)

В сумме **1168 запись**, или примерно 0.9% от исходной выборки. 

In [ ]:
# Текущее количество строк:
df.shape

(127439, 20)

In [ ]:
# Удаляем записи из будущего
filt = (~df["motor_year"].isna()) & (df["motor_year"] <= 2022)
df = df.loc[filt]

df.shape

(127412, 20)

In [ ]:
# Удаляем записи без указания типа трансмиссии
filt = df["transmission_type"].isna()
df = df.loc[~filt]

df.shape

(127391, 20)

In [ ]:
# Удаляем новые авто и авто под заказ
filt = (df["car_state_type"] != "new") & (df["car_state_type"] != "to_order")
df = df[filt]

df.shape

(126851, 20)

In [ ]:
# Удаляем записи с ошибочным объёмом двигателя
filt = ((df["motor_engine_size_litre"] < 0.5) | (df["motor_engine_size_litre"] >= 6.0)) & (df["fuel_type"] != "Електро")
df = df[~filt]

df.shape

(126271, 20)

### Время создания (created_time) конвертируем в timestamp

In [ ]:
from datetime import datetime

def convert_to_unix_time(date_time_string):
    from datetime import datetime
    date_time_object = datetime.strptime(date_time_string, '%Y-%m-%dT%H:%M:%S%z')
    unix_time = int(date_time_object.timestamp())
    return unix_time

df["created_time"] = df["created_time"].apply(lambda t: convert_to_unix_time(t))

df.head()

,id,created_time,car_state_type,cleared_customs,brand,model,motor_year,motor_mileage_thou,car_body,color,transmission_type,drive_type,fuel_type,motor_engine_size_litre,exterior_condition,condition,region,lat,lon,price
0,780257891,1674420147,with_mileage,yes,Opel,corsa,2011.0,170.0,hatchback,6.0,Механічна,front,Дизель,1.3,major_fixes_needed,['after-an-accident'],lv,49.90548,24.09005,100.000000
1,780828675,1674820489,with_mileage,yes,ВАЗ,2106,1992.0,285.0,sedan,3.0,Механічна,back,Бензин,1.6,minor_scratches,['needs_engine_repair'],lv,50.09947,24.34957,81.603786
3,779502018,1673901093,with_mileage,yes,ВАЗ,2101,1979.0,NaN,sedan,1.0,Механічна,back,Бензин,NaN,major_fixes_needed,['needs_body_repair'],dnp,48.40500,35.01300,68.003155
4,780355459,1674493733,with_mileage,yes,ВАЗ,2101,1972.0,500.0,sedan,3.0,Механічна,back,Бензин,1.5,minor_fixes_needed,['fine_condition'],rov,51.33987,26.60507,54.402524
5,776082021,1671384856,with_mileage,yes,ЗАЗ,965,1987.0,100.0,coupe,7.0,Механічна,back,Бензин,1.0,minor_scratches,['needs_body_repair'],vin,48.99579,29.43972,150.000000


### Распарсим столбец "condition" и добавим отдельные столбцы для состояний

In [ ]:
import ast

condition_strings = df["condition"].tolist()
conditions = []

for condition_string in condition_strings:
    condition_list = ast.literal_eval(condition_string)
    for condition in condition_list:
        if condition not in conditions:
            conditions.append(condition)

print("Уникальные значения состояний:\n")
sorted(conditions)

Уникальные значения состояний:



['after-an-accident',
 'fine_condition',
 'first-owner',
 'garage-storage',
 'needs_body_repair',
 'needs_engine_repair',
 'needs_undercarriage_repair',
 'not-bit',
 'not-colored',
 'not-on-the-move',
 'service-book']

In [ ]:
def is_condition_included(cond: str, cond_string: str) -> bool:
  cond_list = ast.literal_eval(cond_string)

  return True if cond in cond_list else False

df["after_an_accident"] = df["condition"].apply(lambda lst: is_condition_included("after-an-accident", lst))
df["fine_condition"] = df["condition"].apply(lambda lst: is_condition_included("fine_condition", lst))
df["first_owner"] = df["condition"].apply(lambda lst: is_condition_included("first-owner", lst))
df["garage_storage"] = df["condition"].apply(lambda lst: is_condition_included("garage-storage", lst))
df["needs_body_repair"] = df["condition"].apply(lambda lst: is_condition_included("needs_body_repair", lst))
df["needs_engine_repair"] = df["condition"].apply(lambda lst: is_condition_included("needs_engine_repair", lst))
df["needs_undercarriage_repair"] = df["condition"].apply(lambda lst: is_condition_included("needs_undercarriage_repair", lst))
df["not_bit"] = df["condition"].apply(lambda lst: is_condition_included("not-bit", lst))
df["not_colored"] = df["condition"].apply(lambda lst: is_condition_included("not-colored", lst))
df["not_on_the_move"] = df["condition"].apply(lambda lst: is_condition_included("not-on-the-move", lst))
df["cleared_customs"] = df["cleared_customs"].apply(lambda x: True if x == "yes" else False)


pd.set_option("max_columns", 100)
df.head()

,id,created_time,car_state_type,cleared_customs,brand,model,motor_year,motor_mileage_thou,car_body,color,transmission_type,drive_type,fuel_type,motor_engine_size_litre,exterior_condition,condition,region,lat,lon,price,after_an_accident,fine_condition,first_owner,garage_storage,needs_body_repair,needs_engine_repair,needs_undercarriage_repair,not_bit,not_colored,not_on_the_move,service_book
0,780257891,1674420147,with_mileage,True,Opel,corsa,2011.0,170.0,hatchback,6.0,Механічна,front,Дизель,1.3,major_fixes_needed,['after-an-accident'],lv,49.90548,24.09005,100.000000,True,False,False,False,False,False,False,False,False,False,False
1,780828675,1674820489,with_mileage,True,ВАЗ,2106,1992.0,285.0,sedan,3.0,Механічна,back,Бензин,1.6,minor_scratches,['needs_engine_repair'],lv,50.09947,24.34957,81.603786,False,False,False,False,False,True,False,False,False,False,False
3,779502018,1673901093,with_mileage,True,ВАЗ,2101,1979.0,NaN,sedan,1.0,Механічна,back,Бензин,NaN,major_fixes_needed,['needs_body_repair'],dnp,48.40500,35.01300,68.003155,False,False,False,False,True,False,False,False,False,False,False
4,780355459,1674493733,with_mileage,True,ВАЗ,2101,1972.0,500.0,sedan,3.0,Механічна,back,Бензин,1.5,minor_fixes_needed,['fine_condition'],rov,51.33987,26.60507,54.402524,False,True,False,False,False,False,False,False,False,False,False
5,776082021,1671384856,with_mileage,True,ЗАЗ,965,1987.0,100.0,coupe,7.0,Механічна,back,Бензин,1.0,minor_scratches,['needs_body_repair'],vin,48.99579,29.43972,150.000000,False,False,False,False,True,False,False,False,False,False,False


### В столбце "color" заменим цветовые коды на текстовые названия цветов

In [ ]:
color_dict = {1: 'Белый', 2: 'Черный', 3: 'Синий', 4: 'Серый', 5: 'Серебристый',
              6: 'Красный', 7: 'Зеленый', 8: 'Апельсин', 9: 'Асфальт', 10: 'Бежевый',
              11: 'Бирюзовый', 12: 'Бронзовый', 13: 'Вишневый', 14: 'Голубой',
              15: 'Желтый', 17: 'Золотой', 18: 'Коричневый', 19: 'Магнолия',
              20: 'Матовый', 21: 'Оливковый', 22: 'Розовый', 23: 'Сафари',
              24: 'Филетовый', 25: 'Хамелеон'}

df['color'] = df['color'].replace(color_dict)


### В столбце "region" заменим цветовые коды регионов на полные названия

In [ ]:
color_dict = {"lv": "Львовская область", 
              "dnp": "Днепропетровская область",
              "rov": "Ровненская область",
              "vin": "Винницкая область",
              "ko": "Киевская область",
              "ter": "Тернопольская область",
              "chn": "Черниговская область",
              "pol": "Полтавская область",
              "sum": "Сумская область",
              "vol": "Волынская область",
              "don": "Донецкая область",
              "chv": "Черновицкая область",
              "nik": "Николаевская область",
              "kha": "Харьковская область",
              "zak": "Закарпатская область",
              "lug": "Луганская область",
              "chk": "Черкасская область",
              "if": "Ивано-Франковская область",
              "od": "Одесская область",
              "kir": "Кировоградская область",
              "khm": "Хмельницкая область",
              "zap": "Запорожская область",
              "zht": "Житомирская область",
              "khe": "Херсонская область",
              "cri": "Автономная Республика Крым"
              }

df['region'] = df['region'].replace(color_dict)
df.head()

,id,created_time,car_state_type,cleared_customs,brand,model,motor_year,motor_mileage_thou,car_body,color,transmission_type,drive_type,fuel_type,motor_engine_size_litre,exterior_condition,condition,region,lat,lon,price,after_an_accident,fine_condition,first_owner,garage_storage,needs_body_repair,needs_engine_repair,needs_undercarriage_repair,not_bit,not_colored,not_on_the_move,service_book
0,780257891,1674420147,with_mileage,True,Opel,corsa,2011.0,170.0,hatchback,Красный,Механічна,front,Дизель,1.3,major_fixes_needed,['after-an-accident'],Львовская область,49.90548,24.09005,100.000000,True,False,False,False,False,False,False,False,False,False,False
1,780828675,1674820489,with_mileage,True,ВАЗ,2106,1992.0,285.0,sedan,Синий,Механічна,back,Бензин,1.6,minor_scratches,['needs_engine_repair'],Львовская область,50.09947,24.34957,81.603786,False,False,False,False,False,True,False,False,False,False,False
3,779502018,1673901093,with_mileage,True,ВАЗ,2101,1979.0,NaN,sedan,Белый,Механічна,back,Бензин,NaN,major_fixes_needed,['needs_body_repair'],Днепропетровская область,48.40500,35.01300,68.003155,False,False,False,False,True,False,False,False,False,False,False
4,780355459,1674493733,with_mileage,True,ВАЗ,2101,1972.0,500.0,sedan,Синий,Механічна,back,Бензин,1.5,minor_fixes_needed,['fine_condition'],Ровненская область,51.33987,26.60507,54.402524,False,True,False,False,False,False,False,False,False,False,False
5,776082021,1671384856,with_mileage,True,ЗАЗ,965,1987.0,100.0,coupe,Зеленый,Механічна,back,Бензин,1.0,minor_scratches,['needs_body_repair'],Винницкая область,48.99579,29.43972,150.000000,False,False,False,False,True,False,False,False,False,False,False


### Удалим столбцы "id", "condition" и "car_state_type", так как они больше не содержат полезной информации

In [ ]:
df.drop(columns=["id", "condition", "car_state_type"], inplace=True)

df.head()

,created_time,cleared_customs,brand,model,motor_year,motor_mileage_thou,car_body,color,transmission_type,drive_type,fuel_type,motor_engine_size_litre,exterior_condition,region,lat,lon,price,after_an_accident,fine_condition,first_owner,garage_storage,needs_body_repair,needs_engine_repair,needs_undercarriage_repair,not_bit,not_colored,not_on_the_move,service_book
0,1674420147,True,Opel,corsa,2011.0,170.0,hatchback,Красный,Механічна,front,Дизель,1.3,major_fixes_needed,Львовская область,49.90548,24.09005,100.000000,True,False,False,False,False,False,False,False,False,False,False
1,1674820489,True,ВАЗ,2106,1992.0,285.0,sedan,Синий,Механічна,back,Бензин,1.6,minor_scratches,Львовская область,50.09947,24.34957,81.603786,False,False,False,False,False,True,False,False,False,False,False
3,1673901093,True,ВАЗ,2101,1979.0,NaN,sedan,Белый,Механічна,back,Бензин,NaN,major_fixes_needed,Днепропетровская область,48.40500,35.01300,68.003155,False,False,False,False,True,False,False,False,False,False,False
4,1674493733,True,ВАЗ,2101,1972.0,500.0,sedan,Синий,Механічна,back,Бензин,1.5,minor_fixes_needed,Ровненская область,51.33987,26.60507,54.402524,False,True,False,False,False,False,False,False,False,False,False
5,1671384856,True,ЗАЗ,965,1987.0,100.0,coupe,Зеленый,Механічна,back,Бензин,1.0,minor_scratches,Винницкая область,48.99579,29.43972,150.000000,False,False,False,False,True,False,False,False,False,False,False


# 3. Заполнение пропусков

### Выведем информацию о пропусках:

In [ ]:
df.isna().sum()

created_time                      0
cleared_customs                   0
brand                             0
model                           223
motor_year                        0
motor_mileage_thou            43120
car_body                          0
color                             0
transmission_type                 0
drive_type                        0
fuel_type                         0
motor_engine_size_litre       50941
exterior_condition                0
region                            0
lat                               0
lon                               0
price                             0
after_an_accident                 0
fine_condition                    0
first_owner                       0
garage_storage                    0
needs_body_repair                 0
needs_engine_repair               0
needs_undercarriage_repair        0
not_bit                           0
not_colored                       0
not_on_the_move                   0
service_book                

### Объединим меткой "another" модели, которые указаны как "drugie" и имеют значение None

In [ ]:
df.loc[(df["model"] == "drugie" ) | (df["model"].isna()), "model"] = "another"

### Для всех электрокаров установим объём двигателя равным 0.
### Записи с другим типом топлива и без указания объёма двигателя заполним медианным значением по модели, если возможно. Иначе - медианным значением объёма по всей выборке

In [ ]:
# Электрокары
df.loc[(df["fuel_type"] == "Електро"), "motor_engine_size_litre"] = 0

# Другие типы топлива
frozen_df = df.copy()
grouped_brand_model = frozen_df.groupby(["brand", "model"])

medians = grouped_brand_model["motor_engine_size_litre"].median()
missing_indices = df[df["motor_engine_size_litre"].isna()].index

for i in missing_indices:
  brand = df.at[i, "brand"]
  model = df.at[i, "model"]

  df.at[i, "motor_engine_size_litre"] = medians[brand, model]

  # try:
  #   df.at[i, "motor_engine_size_litre"] = medians[brand, model]
  # except:
  #   print(df.iloc[i])
  #   break

df.loc[(df["motor_engine_size_litre"].isna()), "motor_engine_size_litre"] = df["motor_engine_size_litre"].median()

### Записи, в которых не указан пробег, заполним медианным пробегом по году выпуска, если возможно. Иначе - средним пробегом по всей выборке

In [ ]:
grouped_year = frozen_df.groupby(["motor_year"])

medians = grouped_year["motor_mileage_thou"].median()
missing_indices = df[df["motor_mileage_thou"].isna()].index

for i in missing_indices:
  year = df.at[i, "motor_year"]
  df.at[i, "motor_mileage_thou"] = medians[year]

df.loc[(df["motor_mileage_thou"].isna()), "motor_mileage_thou"] = df["motor_mileage_thou"].median()

# 4. Вывод информации о данных

Еще раз посмотрим на данные:

In [ ]:
df.shape

(126271, 28)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126271 entries, 0 to 127438
Data columns (total 28 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   created_time                126271 non-null  int64  
 1   cleared_customs             126271 non-null  bool   
 2   brand                       126271 non-null  object 
 3   model                       126271 non-null  object 
 4   motor_year                  126271 non-null  float64
 5   motor_mileage_thou          126271 non-null  float64
 6   car_body                    126271 non-null  object 
 7   color                       126271 non-null  object 
 8   transmission_type           126271 non-null  object 
 9   drive_type                  126271 non-null  object 
 10  fuel_type                   126271 non-null  object 
 11  motor_engine_size_litre     126271 non-null  float64
 12  exterior_condition          126271 non-null  object 
 13  region        

In [ ]:
df.describe()

,created_time,motor_year,motor_mileage_thou,motor_engine_size_litre,lat,lon,price
count,1.262710e+05,126271.000000,126271.000000,126271.000000,126271.000000,126271.000000,126271.000000
mean,1.664626e+09,2003.665782,249.286024,1.855101,49.251972,30.021599,6364.543473
std,1.941432e+07,9.967211,289.991766,0.611016,1.279449,4.136431,6273.675734
min,1.361267e+09,1930.000000,0.000000,0.000000,44.962800,22.160770,27.271141
25%,1.661282e+09,1999.000000,159.000000,1.500000,48.450030,26.134305,2200.000000
50%,1.672248e+09,2006.000000,216.000000,1.800000,49.406110,30.502430,4800.000000
75%,1.675067e+09,2011.000000,269.500000,2.000000,50.356840,33.392000,8450.000000
max,1.677556e+09,2022.000000,3000.000000,5.900000,52.260000,40.129010,100000.000000


In [ ]:
df.head()

,created_time,cleared_customs,brand,model,motor_year,motor_mileage_thou,car_body,color,transmission_type,drive_type,fuel_type,motor_engine_size_litre,exterior_condition,region,lat,lon,price,after_an_accident,fine_condition,first_owner,garage_storage,needs_body_repair,needs_engine_repair,needs_undercarriage_repair,not_bit,not_colored,not_on_the_move,service_book
0,1674420147,True,Opel,corsa,2011.0,170.0,hatchback,Красный,Механічна,front,Дизель,1.3,major_fixes_needed,Львовская область,49.90548,24.09005,100.000000,True,False,False,False,False,False,False,False,False,False,False
1,1674820489,True,ВАЗ,2106,1992.0,285.0,sedan,Синий,Механічна,back,Бензин,1.6,minor_scratches,Львовская область,50.09947,24.34957,81.603786,False,False,False,False,False,True,False,False,False,False,False
3,1673901093,True,ВАЗ,2101,1979.0,145.0,sedan,Белый,Механічна,back,Бензин,1.3,major_fixes_needed,Днепропетровская область,48.40500,35.01300,68.003155,False,False,False,False,True,False,False,False,False,False,False
4,1674493733,True,ВАЗ,2101,1972.0,500.0,sedan,Синий,Механічна,back,Бензин,1.5,minor_fixes_needed,Ровненская область,51.33987,26.60507,54.402524,False,True,False,False,False,False,False,False,False,False,False
5,1671384856,True,ЗАЗ,965,1987.0,100.0,coupe,Зеленый,Механічна,back,Бензин,1.0,minor_scratches,Винницкая область,48.99579,29.43972,150.000000,False,False,False,False,True,False,False,False,False,False,False


# 5. Краткий вывод и сохранение результатов


1.   В данных заполнены пропуски
2.   Удалены столбцы не несущие дополнительной информации
3.   Добавлены новые столбцы состояний авто
4.   Удалены или исправлены выбросы в некоторых столбцах

Данные можно считать готовыми к разведочному анализу

In [ ]:
df.to_csv("cars_preprocessed.csv", index=False)